# Project 1

### Question 1
Estimate σ by computing the standard deviation of the EFFR changes. Report your estimate as a percent.

In [1]:
import pandas as pd

# Load the data file
data = pd.ExcelFile("Project 2.xlsx")

# Parse the first sheet of the Excel file
df = data.parse("Sheet1")

# Calculate the daily changes in the EFFR
df["EFFR_Change"] = df["EFFR"].diff()

# Compute the standard deviation of the EFFR changes to estimate σ
sigma = df["EFFR_Change"].std()

# Convert the standard deviation to a percentage
sigma_percent = sigma * 100

# Print the result
print(f" σ is: {sigma_percent:.2f}%")


 σ is: 168.88%


### Question 2
Estimate κ and θ by minimizing the sum of squares of the difference between the zero-coupon yield predicted by the model and the observed zero-coupon yield. Your each year, you will use the corresponding EFFR as a proxy for rt. Report the RMSE of your estimation and comment on whether the errors are significant or not.

In [2]:
import numpy as np
from scipy.optimize import minimize

# Vasicek model for zero-coupon yields
def vasicek_yield(maturities, r, kappa, theta, sigma):
    """
    maturities: array-like, time to maturity in years
    r: float, current short-term interest rate
    kappa: float, mean-reverting coefficient
    theta: float, long-run interest rate
    sigma: float, volatility of interest rate changes
    """
    xi = (1 - np.exp(-kappa * maturities)) / (kappa * maturities)
    yield_curve = theta * (1 - xi) - (sigma**2 / (2 * kappa**2)) * (1 - 2 * xi + xi**2) + xi * r
    return yield_curve

# Loss function for optimization
def loss_function(params, maturities, observed_yields, short_rates):
    """
    params: list of [kappa, theta]
    maturities: array-like, time to maturity in years
    observed_yields: 2D array, observed zero-coupon yields for all years
    short_rates: 1D array, short-term interest rates (EFFR) for all years
    """
    kappa, theta = params
    sigma = observed_yields.std()
    total_loss = 0
    for i in range(len(short_rates)):
        modeled_yields = vasicek_yield(maturities, short_rates[i], kappa, theta, sigma)
        total_loss += np.sum((observed_yields[i, :] - modeled_yields)**2)
    return total_loss

# Extract maturities and data for all years
maturities = np.array([1, 5, 10, 20, 30])
observed_yields = df[["Y01", "Y05", "Y10", "Y20", "Y30"]].values
short_rates = df["EFFR"].values

# Initial guess for kappa and theta
initial_guess = [0.1, 5.0]

# Minimize the loss function using all data
result = minimize(loss_function, initial_guess, args=(maturities, observed_yields, short_rates), bounds=[(0.01, 2), (0, 10)])

# Optimal parameters
kappa_opt, theta_opt = result.x

# Compute RMSE
sigma_est = observed_yields.std()
rmse = np.sqrt(np.mean([
    np.sum((observed_yields[i, :] - vasicek_yield(maturities, short_rates[i], kappa_opt, theta_opt, sigma_est))**2)
    for i in range(len(short_rates))
]))

# Print results
print(f"Estimated κ (kappa): {kappa_opt:.4f}")
print(f"Estimated θ (theta): {theta_opt:.4f}")
print(f"RMSE: {rmse:.4f}")


Estimated κ (kappa): 0.7291
Estimated θ (theta): 5.7079
RMSE: 2.5641


In [3]:
from scipy.stats import ttest_1samp

# Calculate the residuals
residuals = observed_yields.flatten() - np.array([
    vasicek_yield(maturities, short_rates[i], kappa_opt, theta_opt, sigma_est)
    for i in range(len(short_rates))
]).flatten()

# Perform t-test
t_stat, p_value = ttest_1samp(residuals, 0)

# Print results
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Determine the significance
if p_value < 0.05:
    print("The residuals are significantly different from zero. The model may not fit well.")
else:
    print("The residuals are not significantly different from zero. The model fits well.")


T-statistic: -1.4730
P-value: 0.1433
The residuals are not significantly different from zero. The model fits well.


### Question 3
For each of the years 2000, 2003, 2007, 2013, 2021 and 2023, generate a plot of the term-structure of zero-coupon yields predicted by the model as a function of maturity from 0 to 30 years. For each year, include the observed yields, and comment on how the model captures the observed term-structure of interest rates. In which year(s) the model struggles fitting the term-structure the most?

### Question 4
For the year 2024, generate a plot of the instantaneous futures and forward rates as a function of maturity from 0 to 30 years.

### Question 5
Using the parameters estimated before, and the short-rate as of 11/22/2024, generate a graph of the probability density function of the instantaneous risk-free rate in 1, 5, 20 and 30 years from now.